# Distributed Training using PyTorch

* [Source](https://docs.ray.io/en/latest/train/getting-started-pytorch.html)

* Computing Global Batch Size:
    ```python
    global_batch_size = worker_batch_size * ray.train.get_context().get_world_size()
    ```

In [5]:
import os
import tempfile

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose

import ray.train.torch

* To disable `use_libuv`:
    * Not Working: [Doc](https://pytorch.org/tutorials/intermediate/TCPStore_libuv_backend.html#exit-route-2-add-use-libuv-0-to-init-method-at-processgroup-initialization)
    * Edited [rendezvous.py](D:\Miniconda\envs\mle_proj\Lib\site-packages\torch\distributed\rendezvous.py)  @ 
        * Line 258: `use_libuv` to `use_libuv=False`
    

In [6]:
ray.train.torch.TorchConfig(backend='gloo') # For CPU only
#os.environ["PL_TORCH_DISTRIBUTED_BACKEND"] = "gloo"

Setting,Value
backend,gloo
init_method,env
timeout_s,1800


In [7]:
def train_func():
    # Model, Loss, Optimizer
    model = resnet18(num_classes=10)
    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )

    ## Prepare Model
    model = ray.train.torch.prepare_model(model)
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)

    # Data
    transform = Compose([ToTensor(), 
                         Normalize((0.5,), (0.5,)),
                         ])
    data_dir = os.path.join(tempfile.gettempdir(), "data")
    train_data = FashionMNIST(
        root=data_dir, train=True, download=True, transform=transform
    )
    train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
    train_loader = ray.train.torch.prepare_data_loader(train_loader)

    # Training
    for epoch in range(10):
        if ray.train.get_context().get_world_size() > 1:
            train_loader.sampler.set_epoch(epoch)
        
        for images, labels in train_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        ## Report metrics and checkpoint
        metrics = {"loss": loss.item(), "epoch": epoch}
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            torch.save(
                model.state_dict(),
                os.path.join(temp_checkpoint_dir, "model.pt")
            )
            ray.train.report(
                metrics,
                checkpoint=ray.train.Checkpoint.from_directory(temp_checkpoint_dir)
            )
        
        if ray.train.get_context().get_world_rank() == 0:
            print(metrics)

# Configure scaling and resource requirements. ( Single worker w/ a GPU)
scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)

# Launch distributed training job.
trainer = ray.train.torch.TorchTrainer(
    train_func,
    scaling_config=scaling_config,
)

In [8]:
result = trainer.fit()

2024-11-07 09:35:17,149	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-11-07 09:35:17,167	INFO data_parallel_trainer.py:340 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.


== Status ==
Current time: 2024-11-07 09:35:17 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: D:/TEMP/Temp/ray/session_2024-11-07_09-22-22_314652_6868/artifacts/2024-11-07_09-35-17/TorchTrainer_2024-11-07_09-35-17/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2024-11-07 09:35:22 (running for 00:00:05.12)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: D:/TEMP/Temp/ray/session_2024-11-07_09-22-22_314652_6868/artifacts/2024-11-07_09-35-17/TorchTrainer_2024-11-07_09-35-17/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2024-11-07 09:35:27 (running for 00:00:10.14)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: D:/TEMP/Temp/ray/session_2024-11-07_09-22-22_314652_6868/artifacts

2024-11-07 10:16:55,243	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-11-07 10:16:55,254	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/JCA/ray_results/TorchTrainer_2024-11-07_09-35-17' in 0.0095s.


== Status ==
Current time: 2024-11-07 10:16:55 (running for 00:41:38.09)
Using FIFO scheduling algorithm.
Logical resource usage: 2.0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: D:/TEMP/Temp/ray/session_2024-11-07_09-22-22_314652_6868/artifacts/2024-11-07_09-35-17/TorchTrainer_2024-11-07_09-35-17/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2024-11-07 10:17:05,427	INFO tune.py:1041 -- Total run time: 2508.28 seconds (2498.08 seconds for the tuning loop).
2024-11-07 10:17:05,428	WARNING tune.py:1051 -- Training has been interrupted, but the most recent state was saved.
Resume training with: <FrameworkTrainer>.restore(path="C:/Users/JCA/ray_results/TorchTrainer_2024-11-07_09-35-17", ...)
